<p style="text-align:center">
    <a href="https://skills.network" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo"  />
    </a>
</p>


# **Loading Models and Inference with Hugging Face Inferences**


Estimated time needed: **20** minutes


In this lab, you will explore how to leverage the Hugging Face `transformers` library for various natural language processing (NLP) tasks. You will start by performing text classification and text generation using pretrained models like DistilBERT and GPT-2 without using the `pipeline()` function, understanding the steps involved in loading models, tokenizing input, performing inference, and processing outputs. Then, you will discover the simplicity and efficiency of using the `pipeline()` function to accomplish the same tasks with minimal code. By comparing both approaches, you will appreciate how the `pipeline()` function streamlines the process, making it easier and faster to implement NLP solutions.


## __Table of Contents__

<ol>
    <li><a href="#Objectives">Objectives</a></li>
    <li>
        <a href="#Setup">Setup</a>
        <ol>
            <li><a href="#Installing-required-libraries">Installing required libraries</a></li>
            <li><a href="#Importing-required-libraries">Importing required libraries</a></li>
        </ol>
    </li>
    <li>
        <a href="#Text-classification-with-DistilBERT">Text classification with DistilBERT</a>
    </li>
    <li>
        <a href="#Text-generation-with-GPT-2">Text generation with GPT-2</a>
    </li>
    <li>
        <a href="#Hugging-Face-pipeline()-function">Hugging Face `pipeline()` function</a>
        <ol>
            <li><a href="#Definition">Definition</a></li>
            <li><a href="#Parameters">Parameters</a></li>
            <li><a href="#Task-types">Task types</a>
                <ol>
                    <li><a href="#Example-1:-Text-classification-using-pipeline()">Example 1: Text classification using `pipeline()`</a>
                    </li>
                    <li><a href="#Example-2:-Language-detection-using-pipeline()">Example 2: Language detection using `pipeline()`</a>
                    </li>
                    <li><a href="#Example-3:-Text-generation-using-pipeline()">Example 3: Text generation using `pipeline()`</a>
                    </li>
                    <li><a href="#Example-4:-Text-generation-using-T5-with-pipeline()">Example 4: Text generation using T5 with `pipeline()`</a>
                    </li>
                </ol>
            </li>
            <li><a href="#Benefits-of-using-pipeline()">Benefits of using `pipeline()`</a></li>
            <li><a href="#When-to-use-pipeline()">When to use `pipeline()`</a></li>
            <li><a href="#When-to-avoid-pipeline()">When to avoid `pipeline()`</a></li>
        </ol>
    </li>
    <li>
        <a href="#Exercise:-Fill-mask-task-using-BERT-with-pipeline()">Exercise: Fill-mask task using BERT with `pipeline()`</a>
    </li>
</ol>


## Objectives

- Learn to set up and use the Hugging Face `transformers` library.
- Perform text classification and text generation using DistilBERT and GPT-2 models without `pipeline()`.
- Understand and utilize the `pipeline()` function to simplify various NLP tasks.
- Compare the ease of using models directly versus using the `pipeline()` function.


----


## Setup


For this lab, you will be using the following libraries:

*   [`torch`](https://pytorch.org/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMML0187ENSkillsNetwork31430127-2021-01-01) for deep learning and neural network modeling.
*   [`transformers`](https://huggingface.co/transformers/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMML0187ENSkillsNetwork31430127-2021-01-01) for accessing pretrained models and performing various NLP tasks with ease.


### Installing required libraries


In [ ]:
!pip install torch
!pip install transformers

### Importing required libraries

_It is recommended that you import all required libraries in one place (here):_


In [2]:
from transformers import pipeline
from transformers import DistilBertForSequenceClassification, DistilBertTokenizer
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer

# You can also use this section to suppress warnings generated by your code:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn
warnings.filterwarnings('ignore')

# Text classification with DistilBERT


## Load the model and tokenizer

First, let's initialize a tokenizer and a model for sentiment analysis using DistilBERT fine-tuned on the SST-2 dataset. This setup is useful for tasks where you need to quickly classify the sentiment of a piece of text with a pretrained, efficient transformer model.




In [3]:
# Load the tokenizer and model

tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english")
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english")

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

## Preprocess the input text
Tokenize the input text and convert it to a format suitable for the model:


In [4]:
# Sample text
text = "Congratulations! You've won a free ticket to the Bahamas. Reply WIN to claim."

# Tokenize the input text
inputs = tokenizer(text, return_tensors="pt")

print(inputs)

{'input_ids': tensor([[  101, 23156,   999,  2017,  1005,  2310,  2180,  1037,  2489,  7281,
          2000,  1996, 17094,  1012,  7514,  2663,  2000,  4366,  1012,   102]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}


The token ids are the token indexes  ```attention_mask``` is essential for correctly processing padded sequences, ensuring efficient computation, and maintaining model performance. Even when no tokens are explicitly masked, it helps the model differentiate between actual content and padding, which is critical for accurate and efficient processing of input data.


###  Perform inference
The `torch.no_grad()` context manager is used to disable gradient calculation.
This reduces memory consumption and speeds up computation, as gradients are not needed for inference (i.e. when you are not training the model). The **inputs syntax is used to unpack a dictionary of keyword arguments in Python. In the context of the model(**inputs):


In [5]:
# Perform inference
with torch.no_grad():
    outputs = model(**inputs)

Another method is `input_ids`, and `attention_mask` is their own parameter.


In [ ]:
#model(input_ids=inputs['input_ids'], attention_mask=inputs['attention_mask'])

#### Get the logits
The logits are the raw, unnormalized predictions of the model. Let's extract the logits from the model's outputs to perform further processing, such as determining the predicted class or calculating probabilities.


In [6]:
logits = outputs.logits
logits.shape

torch.Size([1, 2])

## Post-process the output
Convert the logits to probabilities and get the predicted class:


In [7]:
# Convert logits to probabilities
probs = torch.softmax(logits, dim=-1)

# Get the predicted class
predicted_class = torch.argmax(probs, dim=-1)

# Map the predicted class to the label
labels = ["NEGATIVE", "POSITIVE"]
predicted_label = labels[predicted_class]

print(f"Predicted label: {predicted_label}")

Predicted label: POSITIVE


# Text generation with GPT-2


## Load tokenizer
 Load the pretrained GPT-2 tokenizer. The tokenizer is responsible for converting text into tokens that the model can understand.


In [8]:
# Load the tokenizer and model
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

Load the pretrained GPT-2 model with a language modeling head. The model generates text based on the input tokens.


In [9]:
# Load the tokenizer and model

model = GPT2LMHeadModel.from_pretrained("gpt2")

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

## Preprocess the input text  
Tokenize the input text and convert it to a format suitable for the model, like before you have the token indexes, i.e., inputs.


In [10]:
# Prompt
prompt = "Once upon a time"

# Tokenize the input text
inputs = tokenizer(prompt, return_tensors="pt")
inputs

{'input_ids': tensor([[7454, 2402,  257,  640]]), 'attention_mask': tensor([[1, 1, 1, 1]])}

## Perform inference  
Generate text using the model

```inputs:``` Input token IDs from the tokenizer

```attention_mask:``` Mask indicating which tokens to attend to

```pad_token_id:```Padding token ID set to the end-of-sequence token ID

```max_length:``` Maximum length of the generated sequences

```num_return_sequence:``` Number of sequences to generate


In [11]:
# Generate text
output_ids = model.generate(
    inputs.input_ids,
    attention_mask=inputs.attention_mask,
    pad_token_id=tokenizer.eos_token_id,
    max_length=50,
    num_return_sequences=1
)

output_ids

tensor([[7454, 2402,  257,  640,   11,  262,  995,  373,  257, 1295,  286, 1049,
         8737,  290, 1049, 3514,   13,  383,  995,  373,  257, 1295,  286, 1049,
         3514,   11,  290,  262,  995,  373,  257, 1295,  286, 1049, 3514,   13,
          383,  995,  373,  257, 1295,  286, 1049, 3514,   11,  290,  262,  995,
          373,  257]])

or


```python
with torch.no_grad():
    outputs = model(**inputs)

outputs


## Post-process the output  
Decode the generated tokens to get the text:


In [12]:
# Decode the generated text
generated_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)

print(generated_text)

Once upon a time, the world was a place of great beauty and great danger. The world was a place of great danger, and the world was a place of great danger. The world was a place of great danger, and the world was a


# Hugging Face `pipeline()` function

The `pipeline()` function from the Hugging Face `transformers` library is a high-level API designed to simplify the usage of pretrained models for various natural language processing (NLP) tasks. It abstracts the complexities of model loading, tokenization, inference, and post-processing, allowing users to perform complex NLP tasks with just a few lines of code.

## Definition

```python
transformers.pipeline(
    task: str,
    model: Optional = None,
    config: Optional = None,
    tokenizer: Optional = None,
    feature_extractor: Optional = None,
    framework: Optional = None,
    revision: str = 'main',
    use_fast: bool = True,
    model_kwargs: Dict[str, Any] = None,
    **kwargs
)
```

## Parameters

- **task**: `str`
  - The task to perform, such as "text-classification", "text-generation", "question-answering", etc.
  - Example: `"text-classification"`

- **model**: `Optional`
  - The model to use. This can be a string (model identifier from Hugging Face model hub), a path to a directory containing model files, or a pre-loaded model instance.
  - Example: `"distilbert-base-uncased-finetuned-sst-2-english"`

- **config**: `Optional`
  - The configuration to use. This can be a string, a path to a directory, or a pre-loaded config object.
  - Example: `{"output_attentions": True}`

- **tokenizer**: `Optional`
  - The tokenizer to use. This can be a string, a path to a directory, or a pre-loaded tokenizer instance.
  - Example: `"bert-base-uncased"`

- **feature_extractor**: `Optional`
  - The feature extractor to use for tasks that require it (e.g., image processing).
  - Example: `"facebook/detectron2"`

- **framework**: `Optional`
  - The framework to use, either `"pt"` for PyTorch or `"tf"` for TensorFlow. If not specified, it will be inferred.
  - Example: `"pt"`

- **revision**: `str`, default `'main'`
  - The specific model version to use (branch, tag, or commit hash).
  - Example: `"v1.0"`

- **use_fast**: `bool`, default `True`
  - Whether to use the fast version of the tokenizer if available.
  - Example: `True`

- **model_kwargs**: `Dict[str, Any]`, default `None`
  - Additional keyword arguments passed to the model during initialization.
  - Example: `{"output_hidden_states": True}`

- **kwargs**: `Any`
  - Additional keyword arguments passed to the pipeline components.

## Task types

The `pipeline()` function supports a wide range of NLP tasks. Here are some of the common tasks:

1. **Text Classification**: `text-classification`
   - **Purpose**: Classify text into predefined categories.
   - **Use Cases**: Sentiment analysis, spam detection, topic classification.

2. **Text Generation**: `text-generation`
   - **Purpose**: Generate coherent text based on a given prompt.
   - **Use Cases**: Creative writing, dialogue generation, story completion.

3. **Question Answering**: `question-answering`
   - **Purpose**: Answer questions based on a given context.
   - **Use Cases**: Building Q&A systems, information retrieval from documents.

4. **Named Entity Recognition (NER)**: `ner` (or `token-classification`)
   - **Purpose**: Identify and classify named entities (like people, organizations, locations) in text.
   - **Use Cases**: Extracting structured information from unstructured text.

5. **Summarization**: `summarization`
   - **Purpose**: Summarize long pieces of text into shorter, coherent summaries.
   - **Use Cases**: Document summarization, news summarization.

6. **Translation**: `translation_xx_to_yy` (e.g., `translation_en_to_fr`)
   - **Purpose**: Translate text from one language to another.
   - **Use Cases**: Language translation, multilingual applications.

7. **Fill-Mask**: `fill-mask`
   - **Purpose**: Predict masked words in a sentence (useful for masked language modeling).
   - **Use Cases**: Language modeling tasks, understanding model predictions.

8. **Zero-Shot Classification**: `zero-shot-classification`
   - **Purpose**: Classify text into categories without needing training data for those categories.
   - **Use Cases**: Flexible and adaptable classification tasks.

9. **Feature Extraction**: `feature-extraction`
   - **Purpose**: Extract hidden state features from text.
   - **Use Cases**: Downstream tasks requiring text representations, such as clustering, similarity, or further custom model training.


### Example 1: Text classification using `pipeline()`

In this example, you will use the `pipeline()` function to perform text classification. You will load a pretrained text classification model and use it to classify a sample text.

#### Load the text classification model:
We initialize the pipeline for the `text-classification` task, specifying the model `"distilbert-base-uncased-finetuned-sst-2-english"`. This model is fine-tuned for sentiment analysis.

#### Classify the sample text:
We use the classifier to classify a sample text: "Congratulations! You've won a free ticket to the Bahamas. Reply WIN to claim." The `classifier` function returns the classification result, which is then printed.


In [13]:
# Load a general text classification model
classifier = pipeline("text-classification", model="distilbert-base-uncased-finetuned-sst-2-english")

# Classify a sample text
result = classifier("Congratulations! You've won a free ticket to the Bahamas. Reply WIN to claim.")
print(result)

Device set to use cpu


[{'label': 'POSITIVE', 'score': 0.9997586607933044}]


#### Output

The output will be a list of dictionaries, where each dictionary contains:

- `label`: The predicted label (e.g., "POSITIVE" or "NEGATIVE").
- `score`: The confidence score for the prediction.


### Example 2: Language detection using `pipeline()`

In this example, you will use the `pipeline()` function to perform language detection. You will load a pretrained language detection model and use it to identify the language of a sample text.

#### Load the language detection model:
We initialize the pipeline for the `text-classification` task, specifying the model `"papluca/xlm-roberta-base-language-detection"`. This model is fine-tuned for language detection.

#### Classify the sample text:
We use the classifier to detect the language of a sample text: "Bonjour, comment ça va?" The `classifier` function returns the classification result, which is then printed.


In [14]:
from transformers import pipeline

classifier = pipeline("text-classification", model="papluca/xlm-roberta-base-language-detection")
result = classifier("Bonjour, comment ça va?")
print(result)

config.json:   0%|          | 0.00/1.42k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/502 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

Device set to use cpu


[{'label': 'fr', 'score': 0.9934879541397095}]


#### Output
The output will be a list of dictionaries, where each dictionary contains:

- `label`: The predicted language label (e.g., "fr" for French).
- `score`: The confidence score for the prediction.


### Example 3: Text generation using `pipeline()`

In this example, you will use the `pipeline()` function to perform text generation. You will load a pretrained text generation model and use it to generate text based on a given prompt.

#### Initialize the text generation model:
We initialize the pipeline for the `text-generation` task, specifying the model `"gpt2"`. GPT-2 is a well-known model for text generation tasks.


In [15]:
# Initialize the text generation pipeline with GPT-2
generator = pipeline("text-generation", model="gpt2")

Device set to use cpu


#### Generate text based on a given prompt:
We use the generator to generate text based on a prompt: "Once upon a time". Let's specify `max_length=50`, `truncation=True` to limit the generated text to 50 tokens and `num_return_sequences=1` to generate one sequence. The `generator` function returns the generated text, which is then printed.


In [16]:
# Generate text based on a given prompt
prompt = "Once upon a time"
result = generator(prompt, max_length=50, num_return_sequences=1, truncation=True)

# Print the generated text
print(result[0]['generated_text'])

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Once upon a time, if you want a new experience, the company and community should come together, you get to take advantage of any new products you want.

You can always use the community as a forum of discussion to get your thoughts into


#### Output
The output will be a list of dictionaries, where each dictionary contains:

- `generated_text`: The generated text based on the input prompt.


### Example 4: Text generation using T5 with `pipeline()`

In this example, you will use the `pipeline()` function to perform text-to-text generation with the T5 model. You will load a pretrained T5 model and use it to translate a sentence from English to French based on a given prompt.

#### Initialize the text generation model:
We initialize the pipeline for the `text2text-generation task, specifying the model "t5-small". T5 is a versatile model that can perform various text-to-text generation tasks, including translation.


In [17]:
# Initialize the text generation pipeline with T5
generator = pipeline("text2text-generation", model="t5-small")

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

Device set to use cpu


#### Generate text based on a given prompt:
We use the generator to translate a sentence from English to French based on the prompt: "translate English to French: How are you?". Let's specify `max_length=50` to limit the generated text to 50 tokens and `num_return_sequences=1` to generate one sequence. The `generator` function returns the translated text, which is then printed.


In [20]:
# Generate text based on a given prompt
prompt = "translate English to French: How are you?"
result = generator(prompt, max_length=50, num_return_sequences=1)

# Print the generated text
print(result[0]['generated_text'])
print(result)

Comment êtes-vous?
[{'generated_text': 'Comment êtes-vous?'}]


#### Output
The output will be a list of dictionaries, where each dictionary contains:

- `generated_text`: The generated text based on the input prompt.


## Benefits of using `pipeline()`

- **Reduced Boilerplate Code**: Simplifies the code required to perform NLP tasks.
- **Improved Readability**: Makes code more readable and expressive.
- **Time Efficiency**: Saves time by handling model loading, tokenization, inference, and post-processing automatically.
- **Consistent API**: Provides a consistent API across different tasks, allowing for easy experimentation and quick prototyping.
- **Automatic Framework Handling**: Automatically handles the underlying framework (TensorFlow or PyTorch).

## When to use `pipeline()`

- **Quick Prototyping**: When you need to quickly prototype an NLP application or experiment with different models.
- **Simple Tasks**: When performing simple or common NLP tasks that are well-supported by the `pipeline()` function.
- **Deployment**: When deploying NLP models in environments where simplicity and ease of use are crucial.

## When to avoid `pipeline()`

- **Custom Tasks**: When you need to perform highly customized tasks that are not well-supported by the `pipeline()` function.
- **Performance Optimization**: When you need fine-grained control over the model and tokenization process for performance optimization or specific use cases.


# Exercise: Fill-mask task using BERT with `pipeline()`

In this exercise, you will use the `pipeline()` function to perform a fill-mask task using the BERT model. You will load a pretrained BERT model and use it to predict the masked word in a given sentence.


### Instructions

1. **Initialize the fill-mask pipeline** with the BERT model.
2. **Create a prompt** with a masked token.
3. **Generate text** by filling in the masked token.
4. **Print the generated text** with the predictions.


In [19]:
fill_mask = pipeline("fill-mask", model="bert-base-uncased")

prompt = "The capital of France is [MASK]."
result = fill_mask(prompt)

print(result)

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Device set to use cpu


[{'score': 0.4167894423007965, 'token': 3000, 'token_str': 'paris', 'sequence': 'the capital of france is paris.'}, {'score': 0.07141634821891785, 'token': 22479, 'token_str': 'lille', 'sequence': 'the capital of france is lille.'}, {'score': 0.06339266151189804, 'token': 10241, 'token_str': 'lyon', 'sequence': 'the capital of france is lyon.'}, {'score': 0.04444744810461998, 'token': 16766, 'token_str': 'marseille', 'sequence': 'the capital of france is marseille.'}, {'score': 0.030297260731458664, 'token': 7562, 'token_str': 'tours', 'sequence': 'the capital of france is tours.'}]


<details>
    <summary>Click here for Solution</summary>

```python
# Initialize the fill-mask pipeline with BERT
fill_mask = pipeline("fill-mask", model="bert-base-uncased")

# Generate text by filling in the masked token
prompt = "The capital of France is [MASK]."
result = fill_mask(prompt)

# Print the generated text
print(result)
```

</details>


## Authors


[Ashutosh Sagar](https://www.linkedin.com/in/ashutoshsagar/) is completing his MS in CS from Dalhousie University. He has previous experience working with Natural Language Processing and as a Data Scientist.


© Copyright IBM Corporation. All rights reserved.
